In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import cvxopt
np.random.seed(54321)

In [323]:
X_train=pd.read_csv('./data/Xtr.csv', sep=',') #we use this dataset to train our model
Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test=pd.read_csv('./data/Xte.csv', sep=',') #we will use this data set later to validate our model

X_train_mat=pd.read_csv('./data/Xtr_mat100.csv', sep=',') #we use this dataset to train our model
X_test_mat=pd.read_csv('./data/Xte_mat100.csv', sep=',') #we will use this data set later to validate our model

In [3]:
X_train=pd.read_csv('./data/train_data_preprocessing1.csv', sep=',') #we use this dataset to train our model
Y_train=pd.read_csv('./data/Ytr.csv', sep=',') #we use this dataset to train our model
X_test=pd.read_csv('./data/test_data_preprocessing1.csv', sep=',')

In [4]:
# X_test.head()

In [5]:
X_train.tail()

,Id,seq,seq_0,seq_1,seq_2,seq_3,seq_4,seq_5,seq_6,seq_7,...,seq_97,seq_98,seq_99,seq_100,molecular_weight,gravity,iso_electric_point,instability_index,molar_extinction_coefficient,secondary_structure_fraction
1995,1995,TAACTTTTGACAGGTCAGAATACAAAACTGATTTATTTACAGTGTC...,20,1,1,3,20,20,20,20,...,7,1,3,20,8366.2480,0.702970,5.159485,59.928713,562.5,0.174917
1996,1996,ACGCCCATTCCGCCCTGCTAAGCCTCGCCCATTACATCCAGACTGC...,1,3,7,3,3,3,1,20,...,3,3,1,7,8831.7826,1.353465,5.338196,47.579208,1500.0,0.132013
1997,1997,TGGCTACTAGCTAGAGATAGCATCTCTCTGTGGACAACTCTCCAGC...,20,7,7,3,20,1,3,20,...,20,7,7,3,8430.4920,0.716832,5.150330,72.473267,750.0,0.161716
1998,1998,CCCAGCTGTCAAAAAGCAGCCCAAAGGAAGCTCACGGTGTGCCGGC...,3,3,3,1,7,3,20,7,...,20,7,3,3,8705.3152,1.157426,5.350891,47.924752,1187.5,0.145215
1999,1999,TGCTAGTTGATGAAACAATAACTGCTAAAAGGTATACAGCCATGTC...,20,7,3,20,1,7,20,20,...,20,7,20,20,8394.3012,0.746535,5.159485,39.744554,562.5,0.174917


In [252]:
print('The shape of the X_train dataset is:',X_train.shape)
print('The shape of the Y_train dataset is:',Y_train.shape)

The shape of the X_train dataset is: (2000, 2)
The shape of the Y_train dataset is: (2000, 2)


In [253]:
# X_train['len'] = X_train.seq.apply(lambda x : len(x))

In [254]:
X_train.head()

,Id,seq
0,0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...
1,1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...
2,2,GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...
3,3,GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...
4,4,GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...


In [255]:
import string
Alphabet_dict = dict(zip(string.ascii_uppercase, range(1,27)))

In [324]:
#  X_train['seq_1'] = X_train.seq.apply(lambda x : (' '.join(map(str, list(x))))[0])
for i in range(0, 101, 3):
#     X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :Alphabet_dict[x[i]])
#     X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :Alphabet_dict[x[i]])
    
#     X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :Alphabet_dict[x[i]])
#     X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :Alphabet_dict[x[i]]

    X_train['seq_'+str(i)] = X_train.seq.apply(lambda x :x[i:i+3])
    X_test['seq_'+str(i)] = X_test.seq.apply(lambda x :x[i:i+3])



In [352]:
X_train.head()

,Id,seq,seq_0,seq_1,seq_2,seq_3,seq_4,seq_5,seq_6,seq_7,...,seq_97,seq_98,seq_99,seq_100,molecular_weight,gravity,iso_electric_point,instability_index,molar_extinction_coefficient,secondary_structure_fraction
0,0,GAGGGGCTGGGGAGGGGGCTGGCCCAGAGGCACCAGACTCTGCAGA...,7,1,7,7,7,7,3,20,...,7,3,1,7,8082.2632,0.803960,5.370789,72.013861,1000.0,0.181518
1,1,CGGCCTGGGGGCCACATGTGAGTGCTTACCTGTGTGGGGATGAGGG...,3,7,7,3,3,20,7,7,...,7,20,7,7,7733.4276,0.491089,5.416809,52.632673,625.0,0.207921
2,2,GACAACGCCGCTGTCAGCCGCCTTCGACTCACCTGGGAGGTGATGA...,7,1,3,1,1,3,7,3,...,7,3,7,20,8547.0108,0.954455,5.354065,59.237624,1125.0,0.148515
3,3,GCCTCCCTTGGCACCACGGGAGACCAGTTTTGGAGGGGCGGGGCTG...,7,3,3,20,3,3,3,20,...,20,1,1,1,7903.8336,0.684158,5.396423,56.814851,750.0,0.198020
4,4,GCACTACTACACCCATTGCTGTAATAGTAAGTGCCGGTGCCTTCAC...,7,3,1,3,20,1,3,20,...,1,3,3,3,8721.0682,0.910891,5.374207,82.459406,937.5,0.141914


## What about counting number of different caracters 

In [326]:
# A, C, G, T

In [327]:
# X_train.seq.apply(lambda x : np.unique(x[:2]))


In [6]:
X = X_train.drop(['seq', 'Id'], axis=1)
X_t = X_test.drop(['seq', 'Id'], axis=1)
y = Y_train.Bound

## Models

In [7]:
# class LogisticRegressionBinary():
#     def __init__(self, lr=0.1, num_iter=100000, batch_size=1, verbose=False):
#         self.lr = lr
#         self.num_iter = num_iter
#         self.batch_size = batch_size
#         self.verbose = verbose
    
#     def __add_intercept(self, X):
#         intercept = np.ones((X.shape[0], 1))
#         return np.concatenate((intercept, X), axis=1)
    
#     def __sigmoid_func(self, z):
#         return 1 / (1 + np.exp(-z))
    
#     def __loss(self, h, y):
#         return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
#     def fit(self, X, y):
#         y = self.trans_y(y)
#         X = self.__add_intercept(X)
#         self.theta = np.zeros(X.shape[1])
        
#         for i in range(self.num_iter):
#             z = np.dot(X, self.theta)
#             h = self.__sigmoid_func(z)
                        
#             rand = np.random.choice(y.size, self.batch_size).squeeze()
#             gradient = np.dot(X[rand].T, (h[rand] - y[rand]))/y.size   
        
#             self.theta -= self.lr * gradient
#             #print('theta and grad',self.theta.shape ,  gradient.shape )
#             if(self.verbose == True and i % 100 == 0):
#                 z = np.dot(X, self.theta)
#                 h = self.__sigmoid(z)
#                 print(f'loss: {self.__loss(h, y)} \t')
    
#     def predict_probability(self, X):
#         X = self.__add_intercept(X)
    
#         return self.__sigmoid_func(np.dot(X, self.theta))
    
#     def predict(self, X, threshold=.5):
#           return np.where(self.predict_probability(X) >= 0.5, 1, 0)
        
          
#     def Accuracy_check(self,X,y):
#         return np.mean(self.predict(X)==y)
    
#     def trans_y(self, y):
#         if isinstance(y, pd.Series):
#             y = y.values
#         if isinstance(y, list):
#             y = np.array(y)
#         return y

In [8]:
# Ridge Regression (RR)

class solveRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
            
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)

        A = (X.T.dot(X)) + np.eye(p)*lam*n
        b = X.T.dot(y)
        
        self.beta = np.linalg.solve(A, b)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    



# Weighted Ridge Regression (WRR)
class solveWRR():
    def __init__(self, X, y, w, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.w = w
    
    def fit(self):
        
        X = self.X
        y = self.y
        lam = self.lam 
        w = self.w
        
        n, p = X.shape
        assert (len(y) == len(w) == n)

        y1 = np.sqrt(w) * y
        X1 = (np.sqrt(w) * X.T).T
        
        # Hint:
        # Find y1 and X1 such that:
        
        self.beta = solveRR(X1, y1, lam).fit()
                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    

# Logistic Ridge Regression (LRR)
class solveLRR():
    def __init__(self, X, y, lam=0.1):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)
    
        lam = self.lam 
        max_iter = 50
        eps = 1e-3
        sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        
        
        # Initialize
        self.beta = np.zeros(p)

        # Hint: Use IRLS
        for i in range(max_iter):
            beta_old = self.beta
            f = X.dot(beta_old)
            w = sigmoid(f) * sigmoid(-f)
            z = f + y / sigmoid(y*f)
            self.beta = solveWRR(X, z, w, 2*lam).fit()
            # Break condition (achieved convergence)
            #if np.sum((beta-beta_old)**2) < eps:
            #    break                
        return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

In [9]:
# You don't need to look at this, this is just to adapt our matrices
# to the solver being used
solver='cvxopt'
def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
    qp_G = .5 * (P + P.T)   # make sure P is symmetric
    qp_a = -q
    if A is not None:
        qp_C = -np.vstack([A, G]).T
        qp_b = -np.hstack([b, h])
        meq = A.shape[0]
    else:  # no equality constraint
        qp_C = - G.T
        qp_b = - h
        meq = 0
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    solution = cvxopt.solvers.qp(*cvx_matrices)
    return np.array(solution['x']).flatten()

solve_qp = {'quadprog': quadprog_solve_qp, 'cvxopt': cvxopt_qp}[solver]


# SVM primal soft
class svm_primal_soft_to_qp():
    def __init__(self, X, y, C=1):
        self.beta = None
        self.X = X
        self.y = y
        self.w = None
        self.b = None
        self.e = None
        self.C = C
    
    def fit(self):
        
        X = self.X
        y = self.y
        C = self.C
        
        n, p = X.shape
        assert (len(y) == n)

        Xy = np.diag(y).dot(X)
        # Primal formulation, soft margin
        diag_P = np.hstack([np.ones(p), np.zeros(n + 1)])
        # As a regularization, we add epsilon * identity to P
        eps = 1e-12
        diag_P += eps
        P = np.diag(diag_P)

        q = np.hstack([np.zeros(p + 1), C * np.ones(n)])
        # y(wx+b)+ei>=1
        G1 = - np.hstack([Xy, y[:, np.newaxis], np.eye(n)])
        # ei>=0
        G2 = - np.hstack([np.zeros((n, p+1)), np.eye(n)])
        G = np.vstack([G1, G2])
        h = - np.hstack([np.ones(n), np.zeros(n)])
        A = None
        b = None
        
        
        coefs = solve_qp(P, q, G, h, A, b)
        
#         return P, q, G, h, A, b
    
#         self.beta = solveRR(X1, y1, lam).fit()
        
        self.w, self.b, self.e = coefs[:p], coefs[p], coefs[(p+1):]
                
#         return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.w) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    
def rbf_kernel_element_wise(x, y, sigma=1):
    '''
    returns the RBF (Gaussian) kernel k(x, y)
    
    Input:
    ------
    x and y are p-dimensional vectors 
    '''
    K = np.exp(- np.sum((x - y)**2) / (2 * sigma ** 2))
    return K




## Kernel

In [10]:
import numpy as np

### Functions for you to fill in ###

def polynomial_kernel(X, Y, c, p):
    """
        Compute the polynomial kernel between two matrices X and Y::
            K(x, y) = (<x, y> + c)^p
        for each pair of rows x in X and y in Y.

        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            c - a coefficient to trade off high-order and low-order terms (scalar)
            p - the degree of the polynomial kernel

        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    # YOUR CODE HERE
    # raise NotImplementedError
    kernel_matrix = (X.dot(Y.T) + c)**p
    
    return kernel_matrix


def rbf_kernel(X, Y, gamma):
    """
        Compute the Gaussian RBF kernel between two matrices X and Y::
            K(x, y) = exp(-gamma ||x-y||^2)
        for each pair of rows x in X and y in Y.

        Args:
            X - (n, d) NumPy array (n datapoints each with d features)
            Y - (m, d) NumPy array (m datapoints each with d features)
            gamma - the gamma parameter of gaussian function (scalar)

        Returns:
            kernel_matrix - (n, m) Numpy array containing the kernel matrix
    """
    # YOUR CODE HERE
    # raise NotImplementedError
    n, d = X.shape
    m = Y.shape[0]
    
    kernel_matrix = X**2 @ np.ones((d,m)) + np.ones((n,d)) @ Y.T**2 - 2*(X @ Y.T)
    kernel_matrix = np.exp(-gamma*kernel_matrix)
    
    return kernel_matrix


def rbf_kernel_element_wise(x, y, sigma=1):
    '''
    returns the RBF (Gaussian) kernel k(x, y)
    
    Input:
    ------
    x and y are p-dimensional vectors 
    '''
    K = np.exp(- np.sum((x - y)**2) / (2 * sigma ** 2))
    return K

def rbf_kernel(X1, X2, sigma=10):
    '''
    Returns the kernel matrix K(X1_i, X2_j): size (n1, n2)
    
    Input:
    ------
    X1: an (n1, p) matrix
    X2: an (n2, p) matrix
    '''
    # For loop with rbf_kernel_element works but is slow in python
    # Use matrix operations!
    X2_norm = np.sum(X2 ** 2, axis=-1)
    X1_norm = np.sum(X1 ** 2, axis=-1)
    gamma = 1 / (2 * sigma ** 2)
    K = np.exp(- gamma * (X1_norm[:, None] + X2_norm[None, :] - 2 * np.dot(X1, X2.T)))
    return K

def sigma_from_median(X):
    '''
    Returns the median of ||Xi-Xj||
    
    Input
    -----
    X: (n, p) matrix
    '''
    pairwise_diff = X[:, :, None] - X[:, :, None].T
    pairwise_diff *= pairwise_diff
    euclidean_dist = np.sqrt(pairwise_diff.sum(axis=1))
    return np.median(euclidean_dist)

# print(sigma_from_median(X1))

In [11]:
import ipdb

In [31]:
class ksolveRR():
    def __init__(self, X, y, lam= 0.0001, sigma=0.5):
        self.beta = None
        self.X = X
        self.y = y
        self.lam = lam
        self.sigma = sigma
        self.kernel = rbf_kernel
            
    
    def fit(self):
        X = self.X
        y = self.y
        lam = self.lam 
        
        n, p = X.shape
        assert (len(y) == n)
        
        if self.sigma is None:
            self.sigma = sigma_from_median(X)
            
        A = self.kernel(X, X, self.sigma)+n*self.lam*np.eye(n)
        self.alpha = np.linalg.solve(A, y)
        
        return self.beta
    
        
    def predict(self, X, threshold=.5):
        K_x = self.kernel(X, self.X, sigma = self.sigma)
        return np.where(K_x.dot(self.alpha) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

In [13]:
# You don't need to look at this, this is just to adapt our matrices
# to the solver being used
solver='cvxopt'
def quadprog_solve_qp(P, q, G=None, h=None, A=None, b=None):
    qp_G = .5 * (P + P.T)   # make sure P is symmetric
    qp_a = -q
    if A is not None:
        qp_C = -np.vstack([A, G]).T
        qp_b = -np.hstack([b, h])
        meq = A.shape[0]
    else:  # no equality constraint
        qp_C = - G.T
        qp_b = - h
        meq = 0
    return quadprog.solve_qp(qp_G, qp_a, qp_C, qp_b, meq)[0]

def cvxopt_qp(P, q, G, h, A, b):
    P = .5 * (P + P.T)
    cvx_matrices = [
        cvxopt.matrix(M) if M is not None else None for M in [P, q, G, h, A, b] 
    ]
    solution = cvxopt.solvers.qp(*cvx_matrices)
    return np.array(solution['x']).flatten()

solve_qp = {'quadprog': quadprog_solve_qp, 'cvxopt': cvxopt_qp}[solver]


# SVM primal soft
class ksvm_primal_soft_to_qp():
    def __init__(self, X, y, C=1):
        self.beta = None
        self.X = X
        self.y = y
        self.w = None
        self.b = None
        self.e = None
        self.C = C
    
    def fit(self):
        
        X = self.X
        y = self.y
        C = self.C
        
        n, p = X.shape
        assert (len(y) == n)
        
        K = np.exp(-(1/(2*(6.995879868253351)))*np.linalg.norm(X-X)**2)

        Xy = K #np.diag(y).dot(X)
        # Primal formulation, soft margin
        diag_P = np.hstack([np.ones(p), np.zeros(n + 1)])
        # As a regularization, we add epsilon * identity to P
        eps = 1e-12
        diag_P += eps
        P = np.diag(diag_P)

        q = np.hstack([np.zeros(p + 1), C * np.ones(n)])
        # y(wx+b)+ei>=1
        G1 = - np.hstack([Xy, y[:, np.newaxis], np.eye(n)])
        # ei>=0
        G2 = - np.hstack([np.zeros((n, p+1)), np.eye(n)])
        G = np.vstack([G1, G2])
        h = - np.hstack([np.ones(n), np.zeros(n)])
        A = None
        b = None
        
        
        coefs = solve_qp(P, q, G, h, A, b)
        
#         return P, q, G, h, A, b
    
#         self.beta = solveRR(X1, y1, lam).fit()
        
        self.w, self.b, self.e = coefs[:p], coefs[p], coefs[(p+1):]
                
#         return self.beta
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.w) >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)
    


In [14]:
# import ipdb
# # Logistic Ridge Regression (LRR)
# class ksolveRR():
#     def __init__(self, X, y, lam=0.1):
#         self.beta = None
#         self.X = X
#         self.y = y
#         self.lam = lam
        
#     def rbf_kernel(self, X, Y, gamma=0.1):
#         """
#             Compute the Gaussian RBF kernel between two matrices X and Y::
#                 K(x, y) = exp(-gamma ||x-y||^2)
#             for each pair of rows x in X and y in Y.

#             Args:
#                 X - (n, d) NumPy array (n datapoints each with d features)
#                 Y - (m, d) NumPy array (m datapoints each with d features)
#                 gamma - the gamma parameter of gaussian function (scalar)

#             Returns:
#                 kernel_matrix - (n, m) Numpy array containing the kernel matrix
#         """
#         # YOUR CODE HERE
#         # raise NotImplementedError
#         n, d = X.shape
#         m = Y.shape[0]

#         kernel_matrix = X**2 @ np.ones((d,m)) + np.ones((n,d)) @ Y.T**2 - 2*(X @ Y.T)
#         kernel_matrix = np.exp(-gamma*kernel_matrix)

#         return kernel_matrix
    
#     def fit(self):
        
#         X = self.X
#         y = self.y
        
#         n, p = X.shape
#         assert (len(y) == n)
    
#         lam = self.lam 
#         max_iter = 50
#         eps = 1e-3
#         sigmoid = lambda a: 1/(1 + np.exp(-a))
        
        
        
#         # Initialize
#         self.beta = np.zeros(p)
#         ipdb.set_trace()
#         K = self.rbf_kernel(X, X)
        
#         # Hint: Use IRLS
#         for i in range(max_iter):
#             beta_old = self.beta
#             f = K.dot(beta_old)
#             w = sigmoid(f) * sigmoid(-f)
#             z = f + y / sigmoid(y.T.dot(f))
#             self.beta = solveWRR(X, z, w, 2*lam).fit()
#             # Break condition (achieved convergence)
#             #if np.sum((beta-beta_old)**2) < eps:
#             #    break                
#         return self.beta
    
        
#     def predict(self, X, threshold):
#         return np.where(X.dot(self.beta) >= threshold, 1, 0)
        
          
#     def Accuracy_check(self,X, y, threshold=.5):
#         return np.mean(self.predict(X, threshold)==y)

In [15]:
import ipdb
# Logistic Ridge Regression (LRR)
class perceptron():
    def __init__(self, X, y, iteration=100):
        self.theta = np.zeros(len(X))
        self.X = X
        self.y = y
        self.iteration = iteration
        self.theta_0 = 0
    
    def fit(self):
        
        X = self.X
        y = self.y
        
        n, p = X.shape
        assert (len(y) == n)
        
        for it in range(self.iteration):
            for i in range(len(y)):
                if y[i]*(np.dot(self.theta, X[i]) + self.theta_0)<= 0 :
                    self.theta = self.theta + (np.dot(y[i], X[i]))
                    self.theta_0 += y[i]
    
        
    def predict(self, X, threshold):
        return np.where(X.dot(self.self.theta)+self.theta_0 >= threshold, 1, 0)
        
          
    def Accuracy_check(self,X, y, threshold=.5):
        return np.mean(self.predict(X, threshold)==y)

In [16]:
# Kernel Ridge Regression

class KernelRidge():
    '''
    Kernel Ridge Regression
    
    Methods
    ----
    fit
    predict
    '''
    def __init__(self, X, y, lambd=0.1, sigma=None):
        self.kernel = rbf_kernel
        self.sigma = sigma
        self.lambd = lambd
        self.X = X
        self.y = y

    def fit(self):
        
        X = self.X
        y = self.y
        n, p = X.shape
        assert (n == len(y))
    
        self.X_train = X
        
        # Compute default sigma from data
        if self.sigma is None:
            self.sigma = sigma_from_median(X)
        
#         ipdb.set_trace()
        A = self.kernel(X, X, sigma=self.sigma) + n * self.lambd * np.eye(n)
        
        ## self.alpha = (K + n lambda I)^-1 y
        # Solution to A x = y
        self.alpha = np.linalg.solve(A , y)

        return self
        
    def predict(self, X):
        # Prediction rule: 
        K_x = self.kernel(X, self.X_train, sigma=self.sigma)
        return K_x.dot(self.alpha)

# Cross Validation

In [17]:
from sklearn.model_selection import KFold 
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.feature_extraction.text import TransformerM
# from sklearn.feature_extraction.text import TfidfTransformer

In [18]:
# vectorizer = CountVectorizer(max_features=2000)
# vectorizer.fit_transform(X[:10])
# # vectorizer.get_feature_names()

from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)

In [19]:
kfold=KFold(n_splits=5)
onehot_encoder = OneHotEncoder(sparse=False, categories='auto')

# X_cross = X.values
# X_t = X_t.values

y_cross = y.values

# vectorizer = TfidfVectorizer()
# X_cross = vectorizer.fit_transform(X)

X_cross = onehot_encoder.fit_transform(X.values[:, :101])
X_t_enc = onehot_encoder.fit_transform(X_t.values[:, :101])


# X_cross = transformer.fit_transform(X.values)
# X_t_enc = transformer.fit_transform(X_t.values)


# scaler = MinMaxScaler()#MinMaxScaler() # StandardScaler()
# scaler.fit(X_cross)

# X_cross = scaler.transform(X_cross)

In [20]:
X.values.shape

(2000, 107)

In [21]:
X_t.values.shape

(1000, 107)

In [22]:
X_t_enc.shape

(1000, 404)

In [23]:
X_cross.shape

(2000, 404)

In [24]:
from sklearn import linear_model as lm
from sklearn.metrics import accuracy_score, roc_auc_score

# from np.line

In [25]:
# # Simulate data
# np.random.seed(54321)
# p = 2

# def generate_Xy(n_samples, p=101, sigma=.2):
#     # Half of y is 1s, other half is -1
#     y = np.ones(n_samples)
#     mid = int(n_samples / 2)
#     y[mid:] *= -1
    
#     X = np.random.normal(0, sigma, (n_samples, p))
#     X += (1 - y[:, np.newaxis]) / 2 # add 1 when y = -1
#     # X of shape (n, p)
#     # y vector of length n
#     return X, y

# # Training data
# X_train, y_train = generate_Xy(20)

# # Testing data
# X_cross, y_cross = generate_Xy(2000)

# y_cross = np.where(y_cross==1, 1, 0)

In [26]:
# model = KernelRidge(X_cross, y_cross, lambd=0.1, sigma=None)
# y_pred = model.fit().predict(X_cross)
# print(y_pred.shape)

In [27]:
# # Cross validation
# CL = 10 ** np.linspace(0, 2, 100)
# cvsL = np.zeros(len(CL))

# cvsL
# CL

In [28]:
from collections import defaultdict
results = defaultdict(float)

In [508]:
# models = {solveRR: 'Ridge Regression (RR)', solveWRR:'Weighted Ridge Regression (WRR)', \
#           solveLRR : 'Logistic Ridge Regression (LRR)', ksolveRR : 'Kernal Ridge Regression'}

models = {ksolveRR : 'Kernal Ridge Regression'}

lambdas = np.linspace(0, 0.01, 100)
cs = np.linspace(0, 5, 100)

results = {}
for c in cs:
    for lam in lambdas:
        for model in models:
            accuracy = []
            for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):

                X_train, y_train = X_cross[train_index], y_cross[train_index]
                X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]

                if model == solveWRR:
                    w = np.random.rand(len(y_train))
                    model_curr = solveWRR(X_train, y_train, w, lam=0.01)
                else:
        #             model_curr = model(X_train, y_train, lam=0.09)
                    model_curr = model(X_train, y_train, lam, c) #0.00000000001, 4.0103
        #             model_curr = model(X_train, y_train)

                model_curr.fit()

                accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
#                 print(f'accurracy fold {i}: {accuracy[i]}')

            print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)},\
                        with Lambda: {lam} and C: {c}\n')
    
            results[np.mean(accuracy)]=[lam, c]
print(f'Best value gotten : {list(results.keys())[np.argmax(list(results.keys()))]} with\
                        lambda and c : {results[list(results.keys())[np.argmax(list(results.keys()))]]}')

/home/aims/anaconda3/envs/aims/lib/python3.7/site-packages/ipykernel_launcher.py:76: RuntimeWarning: divide by zero encountered in double_scalars
/home/aims/anaconda3/envs/aims/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in multiply
/home/aims/anaconda3/envs/aims/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in greater_equal



Average accuracy Kernal Ridge Regression is : 0.501,                        with Lambda: 0.0 and C: 0.0


Average accuracy Kernal Ridge Regression is : 0.501,                        with Lambda: 0.00010101010101010101 and C: 0.0


Average accuracy Kernal Ridge Regression is : 0.501,                        with Lambda: 0.00020202020202020202 and C: 0.0


Average accuracy Kernal Ridge Regression is : 0.501,                        with Lambda: 0.00030303030303030303 and C: 0.0


Average accuracy Kernal Ridge Regression is : 0.501,                        with Lambda: 0.00040404040404040404 and C: 0.0


Average accuracy Kernal Ridge Regression is : 0.501,                        with Lambda: 0.000505050505050505 and C: 0.0


Average accuracy Kernal Ridge Regression is : 0.501,                        with Lambda: 0.0006060606060606061 and C: 0.0


Average accuracy Kernal Ridge Regression is : 0.501,                        with Lambda: 0.0007070707070707071 and C: 0.0


Average accuracy Kerna

KeyboardInterrupt: 

In [32]:
# models = {solveRR: 'Ridge Regression (RR)', solveWRR:'Weighted Ridge Regression (WRR)', \
#           solveLRR : 'Logistic Ridge Regression (LRR)', ksolveRR : 'Kernal Ridge Regression'}

models = {ksolveRR : 'Kernal Ridge Regression'}
for model in models:
    accuracy = []
    for i, (train_index, validate_index) in enumerate(kfold.split(X_cross)):

        X_train, y_train = X_cross[train_index], y_cross[train_index]
        X_valid, y_valid = X_cross[validate_index], y_cross[validate_index]

        if model == solveWRR:
            w = np.random.rand(len(y_train))
            model_curr = solveWRR(X_train, y_train, w, lam=0.01)
        else:
#             model_curr = model(X_train, y_train, lam=0.09)
            model_curr = model(X_train, y_train, 0.00000000001, 4.0103)
#             model_curr = model(X_train, y_train)
            
        model_curr.fit()

        accuracy.append(model_curr.Accuracy_check(X_valid, y_valid, threshold=0.5))
        print(f'accurracy fold {i}: {accuracy[i]}')
    
    print(f'\nAverage accuracy {models[model]} is : {np.mean(accuracy)}\n')

accurracy fold 0: 0.605
accurracy fold 1: 0.6575
accurracy fold 2: 0.675
accurracy fold 3: 0.6475
accurracy fold 4: 0.6975

Average accuracy Kernal Ridge Regression is : 0.6565



In [447]:
# 0.6535 = 0.68800
# 0.657 = 0.69200

In [448]:
# Cehckinf full model
model = ksolveRR(X_cross, y_cross, 0.00000000001, 4)
# model = svm_primal_soft_to_qp(X_cross, y_cross, C=1)

model.fit()

model.Accuracy_check(X_cross, y_cross, threshold=0.5)

1.0

# Predictions

In [449]:
model = ksolveRR(X_cross, y_cross, 0.00000000001, 4)
model.fit()
y_pred = model.predict(X_t_enc, 0.5)

In [450]:
X = np.arange(1000).reshape(-1, 1)
sample = pd.DataFrame(data=X, columns=['Id'])
sample.head()

,Id
0,0
1,1
2,2
3,3
4,4


In [451]:
sample['Bound'] = y_pred

In [452]:
sample.tail()

,Id,Bound
995,995,0
996,996,0
997,997,0
998,998,1
999,999,1


In [453]:
sample.to_csv('./ksolveRR_65_cv_0.00000000001_4.csv', index=False)